# Language Knowledge (Vocabulary)
Duration: 30 minutes
Content: This section tests your knowledge of Japanese vocabulary, including kanji readings, orthography, word formation, contextually-defined expressions, paraphrases, and usage
It mainly composes following five categories:
- ``Reading Kana`` (Pronunciation Questions): Given a kanji word, choose the correct kana reading.
- `Writing Kanji` (Writing Questions): Given a word written in kana, choose the correct kanji representation.
- `Word Meaning` Selection (Vocabulary Understanding): Choose the most suitable word to fill in the sentence from four options.
- `Synonym Replacement`: Select a word that has the same or similar meaning as the underlined word.
- `Vocabulary Usage`: Assess the usage of words in actual contexts, choosing the most appropriate word usage, including some common Japanese expressions or fixed phrases.

In [16]:
import pandas as pd
import json
import os
import pickle
import re
import uuid
from typing import *
from langchain_openai import AzureOpenAI,AzureChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from langchain_aws import ChatBedrock
from langchain.embeddings.base import Embeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.embeddings import XinferenceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from typing import Annotated, Literal, Sequence
from typing_extensions import TypedDict
from IPython.display import display, Markdown, Latex
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage,RemoveMessage,HumanMessage,AIMessage,ToolMessage
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List, Optional

load_dotenv()

True

In [17]:
# Import N3 Vocabulary
file_path = '../Vocab/n3.csv'
# Read the CSV file
data = pd.read_csv(file_path)
words = data.iloc[:, :2].sample(frac=1).reset_index(drop=True)
# Display the content of the CSV file
words.head()

,expression,reading
0,代金,だいきん
1,同時,どうじ
2,為る,なる
3,交ぜる,まぜる
4,通行,つうこう


#### load Models

In [18]:
# azure_llm = AzureChatOpenAI(
#     azure_endpoint="https://tooldev-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview",
#     api_key=os.environ["AZURE_API_KEY"],
#     model_name="gpt-4o",
#     api_version="2025-01-01-preview",
#     temperature=0.5,
# )

In [19]:
aws_llm = ChatBedrock(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
     # model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0",
    model_kwargs=dict(temperature=0.5),
    region = "us-east-2",
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
)

## Question Explore

# Kanji 读假名（读音问题）

In [20]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display


# Graph state
class QuestionState(TypedDict):
    topic: str
    question: str
    improved_question: str
    final_question: str


kanji_example = """
9. ここから じゅんばん に見てください。
	1.	順番
	2.	項番
	3.	順落
	4.	項落

10. 父は銀行に つとめて います。
	1.	勧めて
	2.	勤めて
	3.	仕めて
	4.	労めて

11. ポケットが さゆう にあるんですね。
	1.	裏表
	2.	右左
	3.	表裏
	4.	左右

12. 昨日の試合は まけて しまいました。
	1.	退けて
	2.	負けて
	3.	失けて
	4.	欠けて

13. かこの 例も調べてみましょう。
	1.	適去
	2.	過古
	3.	過去
	4.	適古

14. この資料はページが ぎゃく になっていますよ。
	1.	達
	2.	変
	3.	逆
	4.	別
"""

print(words)

# Nodes
def generate_question(state: QuestionState):
    """First LLM call to generate initial question"""

    msg = aws_llm.invoke(f"""You are a Japanese teacher. Your job is to write  6 vocabulary questions for candidates to identify the correct kanji writing of a given word in hiragana for a JLPT N3 level exam paper. The question format follows:
Each question presents a word in hiragana within a sentence, and candidates must choose the correct kanji representation from four options. The options should include one correct kanji form and three distractors that are plausible but incorrect. The JLPT exam paper includes a mix of easy, moderate, and difficult questions to accurately assess the test-taker’s proficiency across different aspects of the language.

"""
            f"The vocabulary should be restricted to N3 level, you can refer to the vocabulary in the word list, choosing random words for the questions"
            f"please refer the question examples following the formal exam paper"
            f"append the correct answer and explanation of main challenges and why teacher asks this question to candidate in chinese at each question"
            f"Finally, beautify markdown format"
            f"topic list: {state['topic']}"
            f"word list: {words}"
            f"formal exam paper: {kanji_example}")
    
    return {"question": msg.content}


# Build workflow
kanji_workflow = StateGraph(QuestionState)

# Add nodes
kanji_workflow.add_node("generate_question", generate_question)

# Add edges to connect nodes
kanji_workflow.add_edge(START, "generate_question")
kanji_workflow.add_edge("generate_question", END)

# Compile
kanji_graph = kanji_workflow.compile()


     expression reading
0            代金    だいきん
1            同時     どうじ
2            為る      なる
3           交ぜる     まぜる
4            通行    つうこう
...         ...     ...
2134        罰する    ばっする
2135        始まり    はじまり
2136         突然    とつぜん
2137         問い      とい
2138         礼儀     れいぎ

[2139 rows x 2 columns]


In [21]:
# Show workflow
#display(Image(kanji_graph.get_graph().draw_png()))

In [22]:
# Invoke
kanji = kanji_graph.invoke({"topic": "店で価格を尋ねる | レストランで食べ物を注文する | 交通手段について話す| 天気の状況について話す | 趣味について話す | タクシーを呼ぶ | 目的地を伝える | 飛行機のチケットを予約する | 道に迷ったときに道を尋ねる | 旅行の計画を立てる | 交通機関の遅延について話す | 今日の天気や気温について話す | 季節の変化や服装の選び方を話す | 台風や大雨などの天気予報を確認する | 花粉症の時期について話す"})
display(Markdown(kanji["question"]))

# JLPT N3 Vocabulary Questions - Kanji Selection

## 問題1
レストランで**たの**しく食事をしました。
1. 愉しく
2. 楽しく
3. 悦しく
4. 喜しく

**正解: 2. 楽しく**

*解释: 这道题考察"楽しい"(たのしい)的汉字写法。"楽しい"是N3级别的常用词汇，表示"愉快、开心"。选项1"愉しく"虽然读音相同且意思相近，但在现代日语中较少使用。选项3和4是不存在的写法。出题目的是测试考生对日常生活中常用情感词汇的汉字掌握程度。*

## 問題2
台風が来るので、**てんき**予報をよく確認してください。
1. 大気
2. 空気
3. 天気
4. 気象

**正解: 3. 天気**

*解释: 这道题测试考生对"天気"(てんき)的正确汉字写法的掌握。"天気"是N3级别的基础词汇，表示"天气"。选项4"気象"虽然与天气相关，但读音为"きしょう"。选项1和2分别表示"大气"和"空气"，都不是"てんき"的正确写法。这个词在谈论天气状况时经常使用，是日常会话的重要词汇。*

## 問題3
**こうつう**機関の遅延について駅員に尋ねました。
1. 交通
2. 公通
3. 行通
4. 校通

**正解: 1. 交通**

*解释: 此题考察"交通"(こうつう)的正确汉字写法。"交通"是N3级别的常见词汇，在讨论交通方式和交通工具时经常使用。选项2、3、4都是看起来可能但实际上不正确的写法。教师出这道题的目的是测试考生对日常生活中频繁使用的交通相关词汇的掌握程度。*

## 問題4
**とつぜん**雨が降り始めたので、傘を買いました。
1. 突然
2. 特然
3. 独然
4. 篤然

**正解: 1. 突然**

*解释: 这道题测试"突然"(とつぜん)的正确汉字写法。"突然"是N3级别的重要词汇，表示"突然、忽然"。其他选项虽然读音相似，但都不是正确的汉字组合。在讨论天气变化、意外情况等场景中，"突然"是一个高频使用的词汇，考生需要掌握其正确写法。*

## 問題5
この店の**だいきん**はクレジットカードで支払えますか。
1. 大金
2. 代金
3. 台金
4. 貸金

**正解: 2. 代金**

*解释: 此题考察"代金"(だいきん)的正确汉字写法。"代金"表示"费用、价款"，在购物、询问价格等场景中经常使用。选项1"大金"表示"大量金钱"，选项3和4都不是常用词汇。这道题旨在测试考生在商业交易场景中对价格相关词汇的掌握程度，